_Lambda School Data Science — Big Data_

# AWS SageMaker

### Links

#### AWS
- The Open Guide to Amazon Web Services: EC2 Basics _(just this one short section!)_ https://github.com/open-guides/og-aws#ec2-basics
- AWS in Plain English https://www.expeditedssl.com/aws-in-plain-english
- Amazon SageMaker » Create an Amazon SageMaker Notebook Instance https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html
- Amazon SageMaker » Install External Libraries https://docs.aws.amazon.com/sagemaker/latest/dg/nbi-add-external.html

#### Dask
- Why Dask? https://docs.dask.org/en/latest/why.html
- Use Cases https://docs.dask.org/en/latest/use-cases.html
- User Interfaces https://docs.dask.org/en/latest/user-interfaces.html

#### Numba
- A ~5 minute guide http://numba.pydata.org/numba-doc/latest/user/5minguide.html

## 1. Estimate pi
https://en.wikipedia.org/wiki/Approximations_of_π#Summing_a_circle's_area

### With plain Python

In [1]:
import random

def monte_carlo_pi(nsamples):
    acc = 0
    for _ in range(int(nsamples)):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [2]:
%%time
monte_carlo_pi(1e7)

CPU times: user 4.22 s, sys: 184 µs, total: 4.22 s
Wall time: 4.22 s


3.1412916

### With Numba
http://numba.pydata.org/

In [3]:
from numba import njit

In [4]:
# dramatic time difference just by adding this decorator to code
# njit = nopython-just-in-time
# njit == @jit(nopython=True)
@njit 

def monte_carlo_pi(nsamples):
    acc = 0
    for _ in range(int(nsamples)):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [5]:
%%time
monte_carlo_pi(1e7)

CPU times: user 376 ms, sys: 16.7 ms, total: 392 ms
Wall time: 391 ms


3.1419452

## 2. Loop a slow function

### With plain Python

In [6]:
from time import sleep

def slow_square(x):
    sleep(1) # sleeps for 1 sec
    return x**2

In [7]:
%%time
# 16 s --> confirms that it runs sequentially (one at a time)
[slow_square(n) for n in range(16)]

CPU times: user 1.47 ms, sys: 402 µs, total: 1.87 ms
Wall time: 16 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225]

### With Dask
- https://examples.dask.org/delayed.html
- http://docs.dask.org/en/latest/setup/single-distributed.html

In [10]:
from dask import compute, delayed

In [11]:
# gives 16 functions ready to go
x = delayed((slow_square)(n)for n in range(16))

In [12]:
%%time
compute(x)

CPU times: user 8.11 ms, sys: 27 µs, total: 8.14 ms
Wall time: 6.59 ms


((0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225),)

## 3. Analyze millions of Instacart orders

### Download data
https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2

In [13]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-02-25 20:22:02--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.237.221
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.237.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’

instacart_online_gr 100%[===================>] 196.03M  26.3MB/s    in 13s     

2019-02-25 20:22:15 (15.4 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’ saved [205548478/205548478]



In [14]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [15]:
%cd instacart_2017_05_01

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/instacart_2017_05_01


In [16]:
!ls -lh *.csv

-rw-r--r-- 1 ec2-user ec2-user 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 ec2-user ec2-user  270 May  2  2017 departments.csv
-rw-r--r-- 1 ec2-user ec2-user 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 ec2-user ec2-user  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 ec2-user ec2-user 104M May  2  2017 orders.csv
-rw-r--r-- 1 ec2-user ec2-user 2.1M May  2  2017 products.csv


### With Pandas

#### Load & merge data

In [17]:
import pandas as pd

In [18]:
%%time
order_products = pd.concat([
    pd.read_csv('order_products__prior.csv'), 
    pd.read_csv('order_products__train.csv')])

# this is low end medium sized data -- 33mill+ rows
order_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 0 to 1384616
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 1.3 GB
CPU times: user 9.93 s, sys: 2.04 s, total: 12 s
Wall time: 12 s


In [19]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [20]:
products = pd.read_csv('products.csv')
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
product_id       49688 non-null int64
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [21]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [22]:
%%time
# merging product names above to our previously merged table
# inner join is default, and if ON not specified then intersects columns
order_products = pd.merge(order_products, products[['product_id', 'product_name']])

CPU times: user 7.3 s, sys: 2.07 s, total: 9.37 s
Wall time: 9.35 s


In [23]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,33120,1,1,Organic Egg Whites
1,26,33120,5,0,Organic Egg Whites
2,120,33120,13,0,Organic Egg Whites
3,327,33120,5,1,Organic Egg Whites
4,390,33120,28,1,Organic Egg Whites


#### Most popular products?

In [24]:
%%time
order_products.product_name.value_counts()

CPU times: user 4.06 s, sys: 50.1 ms, total: 4.11 s
Wall time: 4.09 s


Banana                                                                     491291
Bag of Organic Bananas                                                     394930
Organic Strawberries                                                       275577
Organic Baby Spinach                                                       251705
Organic Hass Avocado                                                       220877
Organic Avocado                                                            184224
Large Lemon                                                                160792
Strawberries                                                               149445
Limes                                                                      146660
Organic Whole Milk                                                         142813
Organic Raspberries                                                        142603
Organic Yellow Onion                                                       117716
Organic Garlic  

#### Organic?

In [26]:
%%time
order_products['organic'] = order_products['product_name'].str.contains('Organic')

CPU times: user 17.3 s, sys: 178 ms, total: 17.5 s
Wall time: 17.5 s


In [27]:
%%time
order_products['organic'].value_counts()

CPU times: user 292 ms, sys: 105 ms, total: 397 ms
Wall time: 395 ms


False    23163118
True     10655988
Name: organic, dtype: int64

### With Dask
https://examples.dask.org/dataframe.html

In [28]:
# dask can work with dataframes - not numba
import dask.dataframe as dd # instead of pd for pandas - Here dd for dask dataframe
from dask.distributed import Client

# specifics on how we want dask to run things in parallel
client = Client(n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:40593 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


#### Load & merge data
https://examples.dask.org/dataframes/01-data-access.html#Read-CSV-files

In [29]:
%%time
order_products = dd.read_csv('order_products*.csv')

CPU times: user 18.7 ms, sys: 4.4 ms, total: 23.1 ms
Wall time: 20.9 ms


In [30]:
%%time
order_products = dd.merge(order_products, products[['product_id', 'product_name']])

CPU times: user 28.8 ms, sys: 336 µs, total: 29.2 ms
Wall time: 25.5 ms


In [31]:
%%time
order_products = order_products.persist()

CPU times: user 8.17 ms, sys: 264 µs, total: 8.44 ms
Wall time: 7.3 ms


http://docs.dask.org/en/latest/dataframe-performance.html#persist-intelligently

#### Most popular products?

In [32]:
%%time
order_products['product_name'].value_counts().compute()

CPU times: user 192 ms, sys: 30.2 ms, total: 223 ms
Wall time: 1.37 s


Banana                                                       491291
Bag of Organic Bananas                                       394930
Organic Strawberries                                         275577
Organic Baby Spinach                                         251705
Organic Hass Avocado                                         220877
Organic Avocado                                              184224
Large Lemon                                                  160792
Strawberries                                                 149445
Limes                                                        146660
Organic Whole Milk                                           142813
Organic Raspberries                                          142603
Organic Yellow Onion                                         117716
Organic Garlic                                               113936
Organic Zucchini                                             109412
Organic Blueberries                             

#### Organic?

In [33]:
%%time
order_products['organic'] = order_products['product_name'].str.contains('Organic')

CPU times: user 5.26 ms, sys: 161 µs, total: 5.42 ms
Wall time: 4.86 ms


In [34]:
%%time
# normalize=True NOT enabled in daks
order_products['organic'].value_counts().compute()

CPU times: user 951 ms, sys: 173 ms, total: 1.12 s
Wall time: 8.15 s


False    23163118
True     10655988
Name: organic, dtype: int64

## 4. Fit a machine learning model

### Load data

In [35]:
%cd ../ds1-predictive-modeling-challenge

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/ds1-predictive-modeling-challenge


In [36]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

X_train_numeric = train_features.select_dtypes(np.number)
y_train = train_labels['status_group']

### With 2 cores (like Google Colab)
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [37]:
model = RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=2, random_state=42, verbose=1)
model.fit(X_train_numeric, y_train)
print('Out-of-bag score:', model.oob_score_)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.7s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   11.5s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:   11.7s finished


Out-of-bag score: 0.7206397306397306


### With 16 cores (on AWS m4.4xlarge)

In [40]:
model = RandomForestClassifier(n_estimators=2000, oob_score=True, n_jobs=16, random_state=42, verbose=1)
model.fit(X_train_numeric, y_train)
print('Out-of-bag score:', model.oob_score_)

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    2.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    5.0s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    9.2s
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed:   14.5s
[Parallel(n_jobs=16)]: Done 1768 tasks      | elapsed:   20.9s
[Parallel(n_jobs=16)]: Done 2000 out of 2000 | elapsed:   23.7s finished


Out-of-bag score: 0.723047138047138


## ASSIGNMENT

Revisit a previous assignment or project that had slow speeds or big data.

Make it better with what you've learned today!

You can use `wget` or Kaggle API to get data. Some possibilities include:

- https://www.kaggle.com/c/ds1-predictive-modeling-challenge
- https://www.kaggle.com/ntnu-testimon/paysim1
- https://github.com/mdeff/fma
- https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2 



Also, you can play with [Datashader](http://datashader.org/) and its [example datasets](https://github.com/pyviz/datashader/blob/master/examples/datasets.yml)!